In [1]:
%pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mediapipe as mp
import cv2
import numpy as np
import math
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from datetime import datetime
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import timedelta
from statsmodels.tsa.arima.model import ARIMA
import time
import os


In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [5]:
#Name The Exercise
Exercise = 'squat'

In [115]:
#Opens the camera and record a figure doing an excersice.
data = {"dates" : [],
        "values" : [],
        "spike/deep": []
        }
samples = 0
time.sleep(5)

for root, dirs, files in os.walk('./Squat Downs/'):
    for file in files:
        # Delete each file
        file_path = os.path.join(root, file)
        os.remove(file_path)

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    counter = 0
    iter = 0
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)



            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )

            if Exercise != 'bench':
                # Get coordinates
                R1 = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                R2 = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                R3 = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                #Visibility Extraction
                R_V = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility
                

                L1 = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                L2 = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                L3 = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                #Visibility Extraction
                L_V = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility
                    
            else:
                # Get coordinates
                R1 = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                R2 = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                R3 = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                #Visibility Extraction
                R_V = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
                

                L1 = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                L2 = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                L3 = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                #Visibility Extraction
                L_V = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility + landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
                    
            if R_V > L_V:
                angle = calculate_angle(R1, R2, R3)
            else:
                angle = calculate_angle(L1, L2, L3)
                
            counter+=1

                        # Visualize angle
            cv2.putText(image, str(angle), 
                            #tuple(np.multiply(L_knee, [1280, 720]).astype(int)), 
                            tuple(np.multiply(L2, [640, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA

                                )
            if counter == 4:
                counter = 0
                iter += 1

            """ 
            c = 0
            avg_y = 0
            for i in range(11):
                if landmarks[i+11].visibility > 0.9:
                    avg_y += landmarks[i+11].y
                    c += 1
            avg_y / c
            """
            
            avg_y = (landmarks[11].y + landmarks[13].y + landmarks[15].y)/3

            if Exercise == 'deadlift':
                max_0 = 165
                min_0 = 135
            elif Exercise == 'squat':
                max_0 = 160
                min_0 = 110
            else:
                max_0 = 165
                min_0 = 95



            if angle > max_0:
                data["spike/deep"].append(1)
                data["values"].append(1-avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            elif angle < min_0:
                data["spike/deep"].append(-1)
                data["values"].append(1-avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
                path = './Squat Downs/frame' + str(samples) + '.jpg'
                samples+=1
                cv2.imwrite(path, image)
            else:
                data["spike/deep"].append(0)
                data["values"].append(1-avg_y)
                data["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
        
            
            cv2.imshow('Mediapipe Feed', image)
            
            if ((cv2.waitKey(10) & 0xFF == ord('q')) | (iter == 100)):
                break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.7135549783706665
y: 0.13228312134742737
z: -0.2784648835659027
visibility: 0.9995325803756714
, x: 0.7152662873268127
y: 0.11445781588554382
z: -0.24560129642486572
visibility: 0.9990406632423401
, x: 0.7177804112434387
y: 0.11468395590782166
z: -0.24559161067008972
visibility: 0.998670220375061
, x: 0.7200788259506226
y: 0.11496993899345398
z: -0.2455938309431076
visibility: 0.9982341527938843
, x: 0.7036505341529846
y: 0.11316776275634766
z: -0.27512797713279724
visibility: 0.99931800365448
, x: 0.6975277066230774
y: 0.11240079998970032
z: -0.2751702666282654
visibility: 0.9993895292282104
, x: 0.6908203959465027
y: 0.11150982975959778
z: -0.2751443088054657
visibility: 0.9994446635246277
, x: 0.711942195892334
y: 0.1235525906085968
z: -0.07715880125761032
visibility: 0.9975970387458801
, x: 0.672234833240509
y: 0.11790916323661804
z: -0.21107086539268494
visibility: 0.9990485310554504
, x: 0.7137337327003479
y: 0.15465790033340454
z: -0.21373014152050018
visibility: 0.99671328

In [116]:
sum = 0
for i in range(len(data)-1):
    d1 = datetime.strptime(data["dates"][i], "%H:%M:%S:%f")
    d2 = datetime.strptime(data["dates"][i+1], "%H:%M:%S:%f")
    delta = d2 - d1
    delta_ms = round(delta.microseconds,0) / 1000
    sum = sum + delta_ms
avg_ms = sum // len(data)-1
print(avg_ms)

54.0


In [117]:
for i in range(100):
    #index = 121 - i
    d1 = datetime.strptime(data["dates"][-1], "%H:%M:%S:%f")
    sum = d1 + timedelta(milliseconds=avg_ms)
    data["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
    data["values"].append(0)
    data["spike/deep"].append(0)
    #data["values"].append(data["values"][len(data)-index:][0])
    #data["spike/deep"].append(data["spike/deep"][len(data)-index:][0])

In [118]:
df = pd.DataFrame(data = data)

In [83]:
#To Dataframe
df = pd.DataFrame(data = data)

#df_raw = df_raw.drop(index=range(28))

#without Outliers
#df = df_raw[(df_raw['values'] > df_raw['values'].quantile(q=0.75)) |
#            (df_raw['values'] < df_raw['values'].quantile(q=0.25)) ]

In [119]:
#Export to CSV
df.to_csv("Squats_Unfiltered.csv" , index = False)

In [120]:
#Plot the graph
%matplotlib qt
df.plot(kind='line', y = "values", x = "dates", style='-o')

<AxesSubplot:xlabel='dates'>

In [121]:
#See seasonal components
res = seasonal_decompose(df["values"][:len(df)-100],model = "additive",period=21)
res.plot();

In [122]:
#Autoarima without causal factors
############## NO SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], m=21,trace=True, seasonal=True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[21] intercept   : AIC=inf, Time=2.49 sec
 ARIMA(0,0,0)(0,0,0)[21] intercept   : AIC=-878.522, Time=0.08 sec
 ARIMA(1,0,0)(1,0,0)[21] intercept   : AIC=-2017.276, Time=1.14 sec
 ARIMA(0,0,1)(0,0,1)[21] intercept   : AIC=-1333.253, Time=0.75 sec
 ARIMA(0,0,0)(0,0,0)[21]             : AIC=802.538, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[21] intercept   : AIC=-2006.119, Time=0.14 sec
 ARIMA(1,0,0)(2,0,0)[21] intercept   : AIC=inf, Time=4.48 sec
 ARIMA(1,0,0)(1,0,1)[21] intercept   : AIC=inf, Time=1.66 sec
 ARIMA(1,0,0)(0,0,1)[21] intercept   : AIC=-2018.688, Time=0.59 sec
 ARIMA(1,0,0)(0,0,2)[21] intercept   : AIC=-2016.966, Time=2.15 sec
 ARIMA(1,0,0)(1,0,2)[21] intercept   : AIC=inf, Time=5.41 sec
 ARIMA(0,0,0)(0,0,1)[21] intercept   : AIC=-911.391, Time=0.37 sec
 ARIMA(2,0,0)(0,0,1)[21] intercept   : AIC=-2027.584, Time=0.69 sec
 ARIMA(2,0,0)(0,0,0)[21] intercept   : AIC=-2019.824, Time=0.15 sec
 ARIMA(2,0,0)(1,0,1)[21] intercep

<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  400
Model:             SARIMAX(5, 0, 0)x(0, 0, [1, 2], 21)   Log Likelihood                1027.465
Date:                                 Sat, 27 May 2023   AIC                          -2036.930
Time:                                         21:08:21   BIC                          -2001.007
Sample:                                              0   HQIC                         -2022.704
                                                 - 400                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0322      0.008      3.836      0.000       0.016       0.049
ar.L1          1.1020      0.048     23.137      0.000       1.009       1.195
ar.L2         -0.0558      0.060     -0.926      0.354      -0.174       0.062
ar.L3          0.0131      0.061      0.214      0.831      -0.107       0.133
ar.L4          0.0032      0.060      0.054      0.957      -0.114       0.121
ar.L5         -0.1118      0.041     -2.697      0.007      -0.193      -0.031
ma.S.L21      -0.1071      0.042     -2.554      0.011      -0.189      -0.025
ma.S.L42      -0.0853      0.065     -1.304      0.192      -0.213       0.043
sigma2         0.0003   1.43e-05     23.751      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               438.19
Prob(Q):                              0.95   Prob(JB):                         0.00
Heteroskedasticity (H):               0.31   Skew:                             0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.12
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
"""for i in range(100):
    index = 100 - i
    if data["values"][len(df)-index:][0] == 0:
       data["values"][len(df)-index:][0] = data["values"][len(df)-index-17:][0]
       print(data["values"][len(df)-index-17:][0])
"""

In [123]:
#Use of sarima without CF
############## NO SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:]
model = SARIMAX(train_df["values"], order=(70, 1, 1))
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [124]:
############## NO SPIKES / DEEPS ##################
from statsmodels.tools.eval_measures import rmse
accuracy_no_deeps = 1- rmse(test_df["values"], prediction)
print(accuracy_no_deeps)

0.38617081884843385


In [26]:
#Autoarima with causal factors
############## WITH SPIKES / DEEPS ##################
auto_arima(df["values"][:len(df)-100], exogenous=df["spike/deep"], seasonal=True , start_q=0,max_q= 1, trace  =True).summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=-5482.613, Time=0.15 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=-678.462, Time=0.15 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-2038.584, Time=0.37 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=2158.600, Time=0.05 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=-5539.710, Time=0.59 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=-5700.186, Time=0.36 sec
 ARIMA(5,0,0)(0,0,0)[0] intercept   : AIC=-5721.053, Time=0.80 sec
 ARIMA(5,0,1)(0,0,0)[0] intercept   : AIC=-5705.799, Time=1.63 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept   : AIC=-5701.242, Time=1.32 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=inf, Time=0.34 sec

Best model:  ARIMA(5,0,0)(0,0,0)[0] intercept
Total fit time: 5.990 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1047
Model:               SARIMAX(5, 0, 0)   Log Likelihood                2867.527
Date:                Sun, 21 May 2023   AIC                          -5721.053
Time:                        15:17:59   BIC                          -5686.377
Sample:                             0   HQIC                         -5707.903
                               - 1047                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0131      0.003      4.684      0.000       0.008       0.019
ar.L1          1.2365      0.028     44.815      0.000       1.182       1.291
ar.L2         -0.1783      0.038     -4.732      0.000      -0.252      -0.104
ar.L3          0.2598      0.021     12.285      0.000       0.218       0.301
ar.L4         -0.1905      0.013    -14.229      0.000      -0.217      -0.164
ar.L5         -0.1476      0.009    -17.229      0.000      -0.164      -0.131
sigma2         0.0002   2.35e-06    103.421      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.78   Jarque-Bera (JB):            425758.61
Prob(Q):                              0.38   Prob(JB):                         0.00
Heteroskedasticity (H):               3.14   Skew:                            -4.63
Prob(H) (two-sided):                  0.00   Kurtosis:                       101.36
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [125]:
#Use of sarima with CF
############## WITH SPIKES / DEEPS ##################
train_df = df.iloc[:len(df)-100]
test_df = df.iloc[len(df)-100:] 
model = SARIMAX(endog=train_df["values"], order=(70,1, 0), exog=train_df["spike/deep"])
res = model.fit()
start = len(train_df)
end = len(train_df) + len(test_df) -1
prediction = res.predict(start, end,exog = test_df["spike/deep"]).rename("prediction")
ax = df["values"].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<AxesSubplot:>

In [ ]:
############## WITH SPIKES / DEEPS ##################

# Need to make sure to truncate it in the right place
#prediction

In [126]:
############## WITH SPIKES / DEEPS ##################
from statsmodels.tools.eval_measures import rmse
accuracy = 1- rmse(test_df["values"], prediction)
print(accuracy)

0.3907656371727375


In [127]:
# Old Version Of "calculate_decay_rate()"
def calculate_decay_rate(df):
    decay = []
    #average decay rate
    for i in range(1,len(df) -1 ):
        x1 = df["spike/deep"][i]
        x2 = df["spike/deep"][i+1]
        if (x1 == 1) and (x2 == 0):
            x1_index = i
        elif (x1 == 0)  and (x2 == 1):
            decay.append(x1_index/i)
    decay.append(1)
    return decay


In [128]:
#insert empty (only dates) rows to the data
def insert_missing_rows(df,lead):
    for i in range(lead):
        d = datetime.strptime(df["dates"][-1], "%H:%M:%S:%f")
        sum = d + timedelta(seconds=1)
        df["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
        df["values"].append(0)
    return df


In [129]:
#New Verson Of calculate_decay_rate()
#Calculates the decay insert the results to the dictionary 'decay'.
def calculate_decay_rate(df):
    
    decay = {"dates" : [],
            "values" : []
        }
    try:
        if Exercise == 'deadlift':
            for i in range(1,len(df) -1 ):
                x1 = df["spike/deep"][i]
                x2 = df["spike/deep"][i+1]
                if (x1 == -1) and (x2 == 0):
                    x1_index = i
                elif (x1 == 0)  and (x2 == -1):
                    if len(decay["dates"]) == 0:
                        decay["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))
                    else:
                        d = datetime.strptime(decay["dates"][-1], "%H:%M:%S:%f")
                        sum = d + timedelta(seconds=1)
                        decay["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))
        else:            
            #average decay rate
            for i in range(1,len(df) -1 ):
                x1 = df["spike/deep"][i]
                x2 = df["spike/deep"][i+1]
                if (x1 == 1) and (x2 == 0):
                    x1_index = i
                elif (x1 == 0)  and (x2 == 1):
                    if len(decay["dates"]) == 0:
                        decay["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))
                    else:
                        d = datetime.strptime(decay["dates"][-1], "%H:%M:%S:%f")
                        sum = d + timedelta(seconds=1)
                        decay["dates"].append(datetime.strftime(sum,"%H:%M:%S:%f"))
                        decay["values"].append((len(df) -1) / (i-x1_index))

    except UnboundLocalError as e:
        print("Error: Variable 'x1_index' referenced before assignment.")
        print(f"Exception: {e}")
        

    return decay


In [130]:
#Add one second
def add_sec(date):
    d = datetime.strptime(date, "%H:%M:%S:%f")
    return d + timedelta(seconds=1)


In [131]:
#Adding 3 points between every two points
def add_points(df,lead):

    new = {"dates" : [],
            "values" : []
        }
    for i in range(len(df)-1):
        x1 = df[i]      #left
        x2 = df[i+1]    #right
        m = (x1+x2)/2   #middle
        lm = (x1+m)/2   #left_middle
        rm = (m+x2)/2   #right_middle
        if len(new["dates"]) == 0:
            new["dates"].append(datetime.today().strftime("%H:%M:%S:%f"))
            new["values"].append(x1)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(lm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(m)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(rm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(x2)
        else:
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(lm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(m)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(rm)
            new["dates"].append(datetime.strftime(add_sec(new["dates"][-1]),"%H:%M:%S:%f"))
            new["values"].append(x2)

            

    #Returning the dict populated with the original points + new additional points + forecast points
    return insert_missing_rows(new,lead)

            

In [149]:
#forecast dots to the future
lead = 10
#calc the decay rate of the history

res = calculate_decay_rate(df)
res_df = pd.DataFrame(data = res)

#add the 3 points between 2 points and the lead points to the future
new = add_points(res_df["values"],lead)
new_res_df = pd.DataFrame(data = new)

#Plot the data
ax = new_res_df[5:].plot(legend=True,figsize=(16,8))
ax.set_xlim([5,len(new_res_df)-lead])

res_df = new_res_df

In [150]:
#Auto Arima To the dacay list
train = res_df[1:-lead]
test = res_df[-lead:]

trend = []
for i in range(lead -1):
    trend.append(lead -10 -i)

auto_arima(train["values"], exogenous = trend,trend="t").summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   24
Model:               SARIMAX(1, 1, 0)   Log Likelihood                 -17.732
Date:                Sat, 27 May 2023   AIC                             41.464
Time:                        21:33:24   BIC                             44.870
Sample:                             0   HQIC                            42.321
                                 - 24                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
drift         -0.0028      0.020     -0.140      0.888      -0.042       0.037
ar.L1          0.7288      0.259      2.816      0.005       0.222       1.236
sigma2         0.2648      0.081      3.277      0.001       0.106       0.423
===================================================================================
Ljung-Box (L1) (Q):                   0.26   Jarque-Bera (JB):                25.10
Prob(Q):                              0.61   Prob(JB):                         0.00
Heteroskedasticity (H):               0.15   Skew:                            -1.57
Prob(H) (two-sided):                  0.02   Kurtosis:                         7.05
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [173]:
train["values"].iloc[-4] = train["values"].iloc[-4]*0.8
train["values"].iloc[-3] = train["values"].iloc[-4]*0.8
train["values"].iloc[-2] = train["values"].iloc[-3]*0.8
train["values"].iloc[-1] = train["values"].iloc[-2]*0.8

C:\Users\mosac\AppData\Local\Temp\ipykernel_8080\1935001003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["values"].iloc[-4] = train["values"].iloc[-4]*0.8
C:\Users\mosac\AppData\Local\Temp\ipykernel_8080\1935001003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["values"].iloc[-3] = train["values"].iloc[-4]*0.8
C:\Users\mosac\AppData\Local\Temp\ipykernel_8080\1935001003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [175]:
train["values"].iloc[-1]

5.349962393162395

In [174]:
train["values"].iloc[-4:-1]

21    10.449145
22     8.359316
23     6.687453
Name: values, dtype: float64

In [196]:
from matplotlib import pyplot

model = SARIMAX(endog=train["values"], order=(70,1,1))
model_fit = model.fit()

trend = list(range(lead - 1, -1, -1))

start = len(train["values"])
end = len(train["values"]) + lead
prediction = model_fit.predict(start, end, exogenous = trend,trend="t").rename("prediction")

for i in range(len(prediction)):
    if prediction.iloc[i] < 0:
        prediction.iloc[i] = 0


ax = res_df["values"][1:].plot(legend=True,figsize=(16,8))
prediction.plot(legend=True)

"""
# summary of fit model
print(model_fit.summary())
"""
# line plot of residuals
#residuals = pd.DataFrame(model_fit.resid)
#residuals.plot()

# density plot of residuals
#residuals.plot(kind='kde')
#pyplot.show()
"""
# summary stats of residuals
print(residuals.describe())
"""

c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\mosac\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


'\n# summary stats of residuals\nprint(residuals.describe())\n'

In [194]:
prediction

25    5.397980
26    5.137692
27    5.228390
28    4.676938
29    3.063854
30    1.937243
31    0.031072
32    0.000000
33    0.000000
34    0.000000
35    0.000000
Name: prediction, dtype: float64
